In [1]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data/data12632/


first_round_testin  first_round_traini	submit_example.csv


In [2]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. All changes under this directory will be kept even after reset. Please clean unnecessary files in time to speed up environment loading.
!ls /home/aistudio/work

result.csv


In [3]:
! pip install catboost

Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple/
    100% |████████████████████████████████| 62.4MB 386kB/s ta 0:00:011   24% |████████                        | 15.5MB 4.9MB/s eta 0:00:10    27% |████████▉                       | 17.2MB 3.8MB/s eta 0:00:12    40% |█████████████                   | 25.4MB 3.6MB/s eta 0:00:11    41% |█████████████▍                  | 26.2MB 760kB/s eta 0:00:48    42% |█████████████▌                  | 26.3MB 700kB/s eta 0:00:52    45% |██████████████▌                 | 28.2MB 7.1MB/s eta 0:00:05    71% |██████████████████████▉         | 44.5MB 6.2MB/s eta 0:00:03
    100% |████████████████████████████████| 7.1MB 2.3MB/s ta 0:00:011
  Running setup.py bdist_wheel for retrying ... done
  Stored in directory: /home/aistudio/.cache/pip/wheels/5b/e5/21/797e8ecf814b6ab2399ab254682b20dda1d244183a0e372010
Successfully built retrying


In [4]:
! pip install mlxtend

Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple/
    100% |████████████████████████████████| 1.3MB 9.0MB/s eta 0:00:01
    100% |████████████████████████████████| 13.0MB 2.0MB/s eta 0:00:01   11% |███▉                            | 1.5MB 25.8MB/s eta 0:00:01    24% |████████                        | 3.2MB 61kB/s eta 0:02:39
paddlepaddle-gpu 1.5.1.post97 has requirement matplotlib<=2.2.4; python_version < "3.6", but you'll have matplotlib 3.0.3 which is incompatible.
  Found existing installation: matplotlib 2.2.3
    Uninstalling matplotlib-2.2.3:
      Successfully uninstalled matplotlib-2.2.3


In [5]:
! pip install xgboost

Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple/
    100% |████████████████████████████████| 142.8MB 189kB/s ta 0:00:014   20% |██████▊                         | 30.0MB 4.3MB/s eta 0:00:27    68% |█████████████████████▉          | 97.6MB 3.7MB/s eta 0:00:13    68% |██████████████████████          | 97.8MB 4.3MB/s eta 0:00:11    70% |██████████████████████▍         | 100.1MB 3.8MB/s eta 0:00:12    78% |█████████████████████████▏      | 112.2MB 3.5MB/s eta 0:00:09


In [6]:
! pip install lightgbm

Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple/
    100% |████████████████████████████████| 1.2MB 10.3MB/s ta 0:00:01


In [7]:
# ! pip install tqdm

In [8]:
# ! pip install auto-sklearn

In [9]:
# !pip list


In [10]:
import catboost as cbt
import numpy as np
import pandas as pd
from mlxtend.classifier import StackingClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, roc_auc_score,log_loss
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import fbeta_score, make_scorer
# from tqdm import tqdm
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
import time
import copy
import gc
import os
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [11]:
train_data = pd.read_csv('/home/aistudio/data/data12632/first_round_traini')
test_data = pd.read_csv('/home/aistudio/data/data12632/first_round_testin', encoding='utf-8')
dit = {'Excellent':0,'Good':1,'Pass':2,'Fail':3}
train_data['Quality_label'] = train_data['Quality_label'].map(dit)
labels = pd.get_dummies(train_data['Quality_label']).values
submit = pd.read_csv('/home/aistudio/data/data12632/submit_example.csv')

In [12]:
# 取parameter5-10，取log
features = ['Parameter5', 'Parameter7', 'Parameter8', 'Parameter9', 'Parameter10']
train_data[features] = np.log(train_data[features] + 1e-5)
test_data[features] = np.log(test_data[features] + 1e-5)
# 特征融合
# train_data['5_6'] = train_data['Parameter5'] + train_data['Parameter6']
# test_data['5_6'] = test_data['Parameter5'] + test_data['Parameter6']
train_data['5_7'] = train_data['Parameter5'] + train_data['Parameter7']
test_data['5_7'] = test_data['Parameter5'] + test_data['Parameter7']
train_data['7_8'] = train_data['Parameter7'] + train_data['Parameter8']
test_data['7_8'] = test_data['Parameter7'] + test_data['Parameter8']
train_data['7_9'] = train_data['Parameter7'] + train_data['Parameter9']
test_data['7_9'] = test_data['Parameter7'] + test_data['Parameter9']
train_data['7_10'] = train_data['Parameter7'] + train_data['Parameter10']
test_data['7_10'] = test_data['Parameter7'] + test_data['Parameter10']
train_data['8_9'] = train_data['Parameter8'] + train_data['Parameter9']
test_data['8_9'] = test_data['Parameter8'] + test_data['Parameter9']
train_data['8_10'] = train_data['Parameter8'] + train_data['Parameter10']
test_data['8_10'] = test_data['Parameter8'] + test_data['Parameter10']
train_data['9_10'] = train_data['Parameter9'] + train_data['Parameter10']
test_data['9_10'] = test_data['Parameter9'] + test_data['Parameter10']

train_data['5_10'] = train_data['Parameter5'] + train_data['Parameter10']
test_data['5_10'] = test_data['Parameter5'] + test_data['Parameter10']
# train_data['6_10'] = train_data['Parameter6'] + train_data['Parameter10']
# test_data['6_10'] = test_data['Parameter6'] + test_data['Parameter10']
train_data['7_8_9'] = train_data['Parameter7'] + train_data['Parameter8'] + train_data['Parameter9']
test_data['7_8_9'] = test_data['Parameter7'] + test_data['Parameter8'] + test_data['Parameter9']
train_data['7_9_10'] = train_data['Parameter7'] + train_data['Parameter9'] + train_data['Parameter10']
test_data['7_9_10'] = test_data['Parameter7'] + test_data['Parameter9'] + test_data['Parameter10']
train_data['7_8_10'] = train_data['Parameter7'] + train_data['Parameter8'] + train_data['Parameter10']
test_data['7_8_10'] = test_data['Parameter7'] + test_data['Parameter8'] + test_data['Parameter10']
train_data['8_9_10'] = train_data['Parameter8'] + train_data['Parameter9'] + train_data['Parameter10']
test_data['8_9_10'] = test_data['Parameter8'] + test_data['Parameter9'] + test_data['Parameter10']
train_data['7_8_9_10'] = train_data['Parameter7'] + train_data['Parameter8'] + train_data['Parameter9'] + train_data['Parameter10']
test_data['7_8_9_10'] = test_data['Parameter7'] + test_data['Parameter8'] + test_data['Parameter9'] + test_data['Parameter10']

# train_data['5-6'] = (train_data['Parameter5'] - train_data['Parameter6'])
# test_data['5-6'] = (test_data['Parameter5'] - test_data['Parameter6'])
train_data['7-8'] = (train_data['Parameter7'] - train_data['Parameter8'])
test_data['7-8'] = (test_data['Parameter7'] - test_data['Parameter8'])
train_data['7-9'] = (train_data['Parameter7'] - train_data['Parameter9'])
test_data['7-9'] = (test_data['Parameter7'] - test_data['Parameter9'])
train_data['7-10'] = (train_data['Parameter7'] - train_data['Parameter10'])
test_data['7-10'] = (test_data['Parameter7'] - test_data['Parameter10'])
train_data['8-9'] = (train_data['Parameter8'] - train_data['Parameter9'])
test_data['8-9'] = (test_data['Parameter8'] - test_data['Parameter9'])
train_data['8-10'] = (train_data['Parameter8'] - train_data['Parameter10'])
test_data['8-10'] = (test_data['Parameter8'] - test_data['Parameter10'])
train_data['9-10'] = (train_data['Parameter9'] - train_data['Parameter10'])
test_data['9-10'] = (test_data['Parameter9'] - test_data['Parameter10'])

# train_data['5_6_mul'] = train_data['Parameter5'].mul(train_data['Parameter6'])
# test_data['5_6_mul'] = test_data['Parameter5'].mul(test_data['Parameter6'])
train_data['7_8_mul'] = train_data['Parameter7'].mul(train_data['Parameter8'])
test_data['7_8_mul'] = test_data['Parameter7'].mul(test_data['Parameter8'])
train_data['7_9_mul'] = train_data['Parameter7'].mul(train_data['Parameter9'])
test_data['7_9_mul'] = test_data['Parameter7'].mul(test_data['Parameter9'])
train_data['7_10_mul'] = train_data['Parameter7'].mul(train_data['Parameter10'])
test_data['7_10_mul'] = test_data['Parameter7'].mul(test_data['Parameter10'])
train_data['8_9_mul'] = train_data['Parameter8'].mul(train_data['Parameter9'])
test_data['8_9_mul'] = test_data['Parameter8'].mul(test_data['Parameter9'])
train_data['8_10_mul'] = train_data['Parameter8'].mul(train_data['Parameter10'])
test_data['8_10_mul'] = test_data['Parameter8'].mul(test_data['Parameter10'])
train_data['9_10_mul'] = train_data['Parameter9'].mul(train_data['Parameter10'])
test_data['9_10_mul'] = test_data['Parameter9'].mul(test_data['Parameter10'])

train_data['78_mul_79'] = (train_data['7_8'].mul(train_data['7_9']))
test_data['78_mul_79'] = (test_data['7_8'].mul(test_data['7_9']))
train_data['78_mul_710'] = (train_data['7_8'].mul(train_data['7_10']))
test_data['78_mul_710'] = (test_data['7_8'].mul(test_data['7_10']))
train_data['78_mul_89'] = (train_data['7_8'].mul(train_data['8_9']))
test_data['78_mul_89'] = (test_data['7_8'].mul(test_data['8_9']))
train_data['78_mul_810'] = (train_data['7_8'].mul(train_data['8_10']))
test_data['78_mul_810'] = (test_data['7_8'].mul(test_data['8_10']))
train_data['78_mul_910'] = (train_data['7_8'].mul(train_data['9_10']))
test_data['78_mul_910'] = (test_data['7_8'].mul(test_data['9_10']))
train_data['79_mul_710'] = (train_data['7_9'].mul(train_data['7_10']))
test_data['79_mul_710'] = (test_data['7_9'].mul(test_data['7_10']))
train_data['79_mul_89'] = (train_data['7_9'].mul(train_data['8_9']))
test_data['79_mul_89'] = (test_data['7_9'].mul(test_data['8_9']))
train_data['79_mul_810'] = (train_data['7_9'].mul(train_data['8_10']))
test_data['79_mul_810'] = (test_data['7_9'].mul(test_data['8_10']))
train_data['79_mul_910'] = (train_data['7_9'].mul(train_data['9_10']))
test_data['79_mul_910'] = (test_data['7_9'].mul(test_data['9_10']))
train_data['710_mul_89'] = (train_data['7_10'].mul(train_data['8_9']))
test_data['710_mul_89'] = (test_data['7_10'].mul(test_data['8_9']))
train_data['710_mul_810'] = (train_data['7_10'].mul(train_data['8_10']))
test_data['710_mul_810'] = (test_data['7_10'].mul(test_data['8_10']))
train_data['710_mul_910'] = (train_data['7_10'].mul(train_data['9_10']))
test_data['710_mul_910'] = (test_data['7_10'].mul(test_data['9_10']))
train_data['89_mul_810'] = (train_data['8_9'].mul(train_data['8_10']))
test_data['89_mul_810'] = (test_data['8_9'].mul(test_data['8_10']))
train_data['89_mul_910'] = (train_data['8_9'].mul(train_data['9_10']))
test_data['89_mul_910'] = (test_data['8_9'].mul(test_data['9_10']))
train_data['810_mul_910'] = (train_data['8_10'].mul(train_data['9_10']))
test_data['810_mul_910'] = (test_data['8_10'].mul(test_data['9_10']))

train_data['5_8'] = train_data['Parameter5'] + train_data['Parameter8']
test_data['5_8'] = test_data['Parameter5'] + test_data['Parameter8']
train_data['7+8*8'] = train_data['Parameter7'] + train_data['Parameter8'] * train_data['Parameter8']
test_data['7+8*8'] = test_data['Parameter7'] + test_data['Parameter8'] * test_data['Parameter8']
train_data['7*7+8'] = train_data['Parameter7'] * train_data['Parameter7'] + train_data['Parameter8']
test_data['7*7+8'] = test_data['Parameter7'] * test_data['Parameter7'] + test_data['Parameter8']

train_data['9/7'] = train_data['Parameter9'] / train_data['Parameter7']
test_data['9/7'] = test_data['Parameter9'] / test_data['Parameter7']
train_data['9/8'] = train_data['Parameter9'] / train_data['Parameter8']
test_data['9/8'] = test_data['Parameter9'] / test_data['Parameter8']
train_data['1/7'] = 1. / train_data['Parameter7']
test_data['1/7'] = 1. / test_data['Parameter7']
train_data['9%8'] = train_data['Parameter9'] % train_data['Parameter8']
test_data['9%8'] = test_data['Parameter9'] % test_data['Parameter8']
train_data['7%9'] = train_data['Parameter7'] % train_data['Parameter9']
test_data['7%9'] = test_data['Parameter7'] % test_data['Parameter9']
train_data['1/9'] = 1. / train_data['Parameter9']
test_data['1/9'] = 1. / test_data['Parameter9']
train_data['7/8'] = train_data['Parameter7'] / train_data['Parameter8']
test_data['7/8'] = test_data['Parameter7'] / test_data['Parameter8']
train_data['8%9'] = train_data['Parameter8'] % train_data['Parameter9']
test_data['8%9'] = test_data['Parameter8'] % test_data['Parameter9']
train_data['7/9'] = train_data['Parameter7'] / train_data['Parameter9']
test_data['7/9'] = test_data['Parameter7'] / test_data['Parameter9']
train_data['8/7'] = train_data['Parameter8'] / train_data['Parameter7']
test_data['8/7'] = test_data['Parameter8'] / test_data['Parameter7']
train_data['1/8'] = 1. / train_data['Parameter8']
test_data['1/8'] = 1. / test_data['Parameter8']
train_data['8%7'] = train_data['Parameter8'] % train_data['Parameter7']
test_data['8%7'] = test_data['Parameter8'] % test_data['Parameter7']
train_data['8+9/8%9'] = (train_data['Parameter8'] + train_data['Parameter9']) / train_data['Parameter8'] % train_data['Parameter9']
test_data['8+9/8%9'] = (test_data['Parameter8'] + test_data['Parameter9']) / test_data['Parameter8'] % test_data['Parameter9']
train_data['7%8'] = train_data['Parameter7'] % train_data['Parameter8']
test_data['7%8'] = test_data['Parameter7'] % test_data['Parameter8']
train_data['8/9'] = train_data['Parameter8'] / train_data['Parameter9']
test_data['8/9'] = test_data['Parameter8'] / test_data['Parameter9']

train_data['7+8%7+9'] = train_data['Parameter7'] + train_data['Parameter8'] % train_data['Parameter7'] + train_data['Parameter9']
test_data['7+8%7+9'] = test_data['Parameter7'] + test_data['Parameter8'] % test_data['Parameter7'] + test_data['Parameter9']
train_data['1/8/9'] = 1./train_data['Parameter8']/train_data['Parameter9']
test_data['1/8/9'] = 1./test_data['Parameter8']/test_data['Parameter9']
train_data['9%7/9'] = train_data['Parameter9']%train_data['Parameter7']/train_data['Parameter9']
test_data['9%7/9'] = test_data['Parameter9']%test_data['Parameter7']/test_data['Parameter9']
train_data['8+9/8*9'] = train_data['Parameter8']+train_data['Parameter9']/train_data['Parameter8']*train_data['Parameter9']
test_data['8+9/8*9'] = test_data['Parameter8']+test_data['Parameter9']/test_data['Parameter8']*test_data['Parameter9']
train_data['9%7/7*8'] = train_data['Parameter9']%train_data['Parameter7']/train_data['Parameter7']*train_data['Parameter8']
test_data['9%7/7*8'] = test_data['Parameter9']%test_data['Parameter7']/test_data['Parameter7']*test_data['Parameter8']
train_data['7%8/7'] = train_data['Parameter7']%train_data['Parameter8']/train_data['Parameter7']
test_data['7%8/7'] = test_data['Parameter7']%test_data['Parameter8']/test_data['Parameter7']
train_data['9%8*9'] = train_data['Parameter9']%train_data['Parameter8']*train_data['Parameter9']
test_data['9%8*9'] = test_data['Parameter9']%test_data['Parameter8']*test_data['Parameter9']
train_data['8*9%7'] = train_data['Parameter8']*train_data['Parameter9']%train_data['Parameter7']
test_data['8*9%7'] = test_data['Parameter8']*test_data['Parameter9']%test_data['Parameter7']

train_data['8%9*8'] = train_data['Parameter8']%train_data['Parameter9']*train_data['Parameter8']
test_data['8%9*8'] = test_data['Parameter8']%test_data['Parameter9']*test_data['Parameter8']
train_data['7*8*9'] = train_data['Parameter8']*train_data['Parameter7']*train_data['Parameter9']
test_data['7*8*9'] = test_data['Parameter8']*test_data['Parameter7']*test_data['Parameter9']
train_data['8+9/7*8']=train_data['Parameter8']+train_data['Parameter9']/train_data['Parameter7']*train_data['Parameter9']
test_data['8+9/7*8']=test_data['Parameter8']+test_data['Parameter9']/test_data['Parameter7']*test_data['Parameter9']
train_data['8%7/(9%8)'] = train_data['Parameter8']%train_data['Parameter7']/(train_data['Parameter9']%train_data['Parameter8'])
test_data['8%7/(9%8)'] = test_data['Parameter8']%test_data['Parameter7']/(test_data['Parameter9']%test_data['Parameter8'])
train_data['7%8/(8%9)'] = train_data['Parameter7']%train_data['Parameter8']/(train_data['Parameter8']%train_data['Parameter9'])
test_data['7%8/(8%9)'] = test_data['Parameter7']%test_data['Parameter8']/(test_data['Parameter8']%test_data['Parameter9'])
train_data['8+9/(9%8)']=train_data['Parameter8']+train_data['Parameter9']/(train_data['Parameter9']%train_data['Parameter8'])
test_data['8+9/(9%8)']=test_data['Parameter8']+test_data['Parameter9']/(test_data['Parameter9']%test_data['Parameter8'])
train_data['8*(9%7)+8'] = train_data['Parameter8']*(train_data['Parameter9']%train_data['Parameter7']+train_data['Parameter8'])
test_data['8*(9%7)+8'] = test_data['Parameter8']*(test_data['Parameter9']%test_data['Parameter7']+test_data['Parameter8'])
train_data['7+8*9'] = train_data['Parameter7']+train_data['Parameter8']*train_data['Parameter9']
test_data['7+8*9'] = test_data['Parameter7']+test_data['Parameter8']*test_data['Parameter9']
train_data['7%8/(9%8)'] = train_data['Parameter7']%train_data['Parameter8']/(train_data['Parameter9']%train_data['Parameter8'])
test_data['7%8/(9%8)'] = test_data['Parameter7']%test_data['Parameter8']/(test_data['Parameter9']%test_data['Parameter8'])
train_data['9%8/(7%8)'] = train_data['Parameter9']%train_data['Parameter8']/(train_data['Parameter7']%train_data['Parameter8'])
test_data['9%8/(7%8)'] = test_data['Parameter9']%test_data['Parameter8']/(test_data['Parameter7']%test_data['Parameter8'])
train_data['5%8'] = train_data['Parameter5']%train_data['Parameter8']
test_data['5%8'] = test_data['Parameter5']%test_data['Parameter8']
# train_data['6%8'] = train_data['Parameter6']%train_data['Parameter8']
# test_data['6%8'] = test_data['Parameter6']%test_data['Parameter8']
train_data['10%8'] = train_data['Parameter10']%train_data['Parameter8']
test_data['10%8'] = test_data['Parameter10']%test_data['Parameter8']

train_data['7*8'] = train_data['Parameter7'] * train_data['Parameter8']
test_data['7*8'] = test_data['Parameter7'] * test_data['Parameter8']
train_data['5*5'] = train_data['Parameter5'] * train_data['Parameter5']
test_data['5*5'] = test_data['Parameter5'] * test_data['Parameter5']
# train_data['6*6'] = train_data['Parameter6'] * train_data['Parameter6']
# test_data['6*6'] = test_data['Parameter6'] * test_data['Parameter6']
train_data['7*7'] = train_data['Parameter7'] * train_data['Parameter7']
test_data['7*7'] = test_data['Parameter7'] * test_data['Parameter7']
train_data['8*8'] = train_data['Parameter8'] * train_data['Parameter8']
test_data['8*8'] = test_data['Parameter8'] * test_data['Parameter8']
train_data['9*9'] = train_data['Parameter9'] * train_data['Parameter9']
test_data['9*9'] = test_data['Parameter9'] * test_data['Parameter9']
train_data['10*10'] = train_data['Parameter10'] * train_data['Parameter10']
test_data['10*10'] = test_data['Parameter10'] * test_data['Parameter10']

scale_features = [
    # '5_6',
    '7_9', '8_9', '7_8', '8_10', '9_10', '7_10', '9%8', '7/8', '7%8', '8/9',
				  ]


# 组合所有特征
features = features + scale_features

print('all train features:', features)

for fc in features:
	n = train_data['{}'.format(fc)].nunique()
# 	这个的左右就是显示特征中的唯一值
	print(fc + ':', n)

all train features: ['Parameter5', 'Parameter7', 'Parameter8', 'Parameter9', 'Parameter10', '7_9', '8_9', '7_8', '8_10', '9_10', '7_10', '9%8', '7/8', '7%8', '8/9']
Parameter5: 132
Parameter7: 14
Parameter8: 23
Parameter9: 16
Parameter10: 41
7_9: 49
8_9: 52
7_8: 43
8_10: 279
9_10: 260
7_10: 291
9%8: 49
7/8: 43
7%8: 41
8/9: 52


In [13]:
# train_data.shape

In [14]:
# train_data.sort_values('Parameter2', ascending=False)[:3]

In [25]:
# --------------------------------------------------------
# train
# --------------------------------------------------------
x_train = train_data[features].values
x_test = test_data[features].values

def lightgbm_model():
	model = LGBMClassifier(max_depth=7, learning_rate=0.01, n_estimators=1000, num_leaves=16,
						   objective='multiclass', silent=True,
						   reg_lambda=4., #reg_alpha=1.,
						   #bagging_fraction=0.9, feature_fraction=0.9,
						   )
	return model

def xgboost_model():
	model = XGBClassifier(max_depth=5, n_estimators=1000, learning_rate=0.01, silent=True,min_child_weight=4,colsample_bytree=0.5,
						  #objective='multi:softmax'
						  )
	return model

def catboost_model():
	cbt_model = cbt.CatBoostClassifier(#iterations=100000, learning_rate=0.01, verbose=0,
		iterations=3000, learning_rate=0.01, verbose=0,
		max_depth=7, #reg_lambda=5.,
		task_type='GPU',   # cat_features=cat_list,
		loss_function='MultiClass',
# 		l2_leaf_reg = 7
		)
	return cbt_model

def catboost_importance():
	model = catboost_model()
	model.fit(x_train, np.argmax(labels, 1))
	importance = model.get_feature_importance(prettified=True)
# 	这个其实就是类似PCA一样，找到特征的贡献
	print(importance.columns)
	for i in range(len(features)):
		print(features[int(importance['Feature Id'][i])] + ' :', importance['Importances'][i])

In [16]:
def grid_search(mode):
	def my_custom_loss_func(y_true, y_pred):
		diff = np.mean(-y_true * np.log(y_pred + 1e-5))
		return diff
	score = make_scorer(my_custom_loss_func, greater_is_better=False)
	
	# grid search for lightgbm
	if mode=='lgb':
		parameters = {'max_depth': [5, 7, 9, 11], 'num_leaves ': [16, 32, 64], 'reg_lambda': [10, 5, 3, 1, 0.1]}
		model = lightgbm_model()
	# grid search for catboost
	if mode=='cat':
		parameters = {'iterations': [3000, 4000, 5000],'max_depth': [ 6, 7, 8, 9],'learning_rate':[0.01, 0.008,0.005]}
		model = catboost_model()
	# grid search for xgboost
	if mode=='xgb':
		parameters = {'max_depth': [5, 7, 9, 11],'n_estimators':[1000, 2000, 3000], 
		'min_child_weight' : [1,2,3,4],'colsample_bytree': [0.5,0.8]}
		model = xgboost_model()
	# search
	clf = GridSearchCV(model, parameters, cv=5, scoring=score, verbose=2)
	clf.fit(x_train, np.argmax(labels, 1))
	print('best parameters:', clf.best_params_)
	print('best score:', clf.best_score_)
	

In [26]:
catboost_importance()

Index(['Feature Id', 'Importances'], dtype='object')
8_10 : 19.402833378351282
Parameter5 : 19.322216067015155
Parameter10 : 14.06220285234933
7_10 : 9.196922160206949
9_10 : 8.360164246390838
7/8 : 5.8836887909739435
7_9 : 4.211004342387408
8/9 : 3.713413454954743
9%8 : 3.3350507894628154
8_9 : 3.3083812161438404
7_8 : 3.0073011782551378
7%8 : 2.1841446318003506
Parameter9 : 1.707117174762718
Parameter8 : 1.2927576141301267
Parameter7 : 1.0128021028153442


In [27]:
# k-fold train
def kfold_train(mode):
	acc_list, loss_list = [], []
	prediction = np.zeros((x_test.shape[0], 4))
	for i in range(10):
		print(str(i+1) + ' th kflod' + '*'*50)
		kf = KFold(n_splits=5, shuffle=True, random_state=i)
		kfold_list = []
		for k, (train_index, test_index) in enumerate(kf.split(x_train)):
			print(str(k+1) + 'fold--------------')
			train_x, train_y = x_train[train_index], labels[train_index]
			test_x, test_y = x_train[test_index], labels[test_index]
			# train
			if mode == 'cat':
				model = catboost_model()
				model.fit(train_x, np.argmax(train_y, 1), eval_set=(test_x, np.argmax(test_y, 1)),
					  #early_stopping_rounds=1000, verbose=False
						)
				
				#print(pd.DataFrame({'column': features, 'importance': model.feature_importances_}).sort_values(by='importance'))
			if mode == 'lgb':
				model = lightgbm_model()
				model.fit(train_x, np.argmax(train_y, 1), eval_set=(test_x, np.argmax(test_y, 1)),
					  # early_stopping_rounds=50, verbose=True
						  verbose=False
						  )
			if mode == 'xgb':
				model = xgboost_model()
				model.fit(train_x, np.argmax(train_y, 1), verbose=True)
			if mode == 'sclf':
			    model = StackingClassifier(
			        classifiers=[catboost_model(),xgboost_model(), lightgbm_model()],
			        use_probas = True,average_probas = False,meta_classifier = lr)
			    model.fit(train_x, np.argmax(train_y, 1))
			# test
			pred = model.predict_proba(test_x)
			acc = accuracy_score(np.argmax(test_y, 1), np.argmax(pred, 1))
			loss = log_loss(test_y, pred)
			acc_list.append(acc)
			loss_list.append(loss)
			kfold_list.append(loss)
			print('test acc: %f, test loss: %f' % (acc, loss))
			# predict
			prediction += model.predict_proba(x_test)
# 			返回的是概率
		print('this fold mean loss:', np.mean(kfold_list))
	print('*'*50)
	print('mean acc: %f, mean loss: %f' % (np.mean(acc_list), np.mean(loss_list)))
	prediction = prediction / 50.
	return prediction

In [41]:
from decimal import Decimal
def submit_result(prediction):
	sub = test_data[['Group']]
	prob_cols = [i for i in submit.columns if i not in ['Group']]
	for i, f in enumerate(prob_cols):
		sub[f] = prediction[:, i]
	for i in prob_cols:
		sub[i] = sub.groupby('Group')[i].transform('mean')
	sub = sub.drop_duplicates()
	for i in prob_cols:
	    print(sub[i])
	    sub[i] = sub[i].apply(lambda x:(x//0.02+1)*0.02 if Decimal(str(x))%Decimal(str(0.02)) > 0.01 else x)
	    sub[i] = sub[i].apply(lambda x:(x//0.02)*0.02 if Decimal(str(x))%Decimal(str(0.02)) < 0.01 else x)
	sub.to_csv("/home/aistudio/work/result.csv", index=False)

time1 = time.clock()
prediction = kfold_train('sclf')
time2 = time.clock()
print('running time: ', str((time2 - time1)/60))
submit_result(prediction)

1 th kflod**************************************************
1fold--------------


/opt/conda/envs/python35-paddle120-env/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/envs/python35-paddle120-env/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


test acc: 0.529167, test loss: 1.105401
2fold--------------


/opt/conda/envs/python35-paddle120-env/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/envs/python35-paddle120-env/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


test acc: 0.526667, test loss: 1.119847
3fold--------------


...

test acc: 0.525833, test loss: 1.117572
5fold--------------


/opt/conda/envs/python35-paddle120-env/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/envs/python35-paddle120-env/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


test acc: 0.537500, test loss: 1.103709
this fold mean loss: 1.1203098434585672
**************************************************
mean acc: 0.527600, mean loss: 1.121543
running time:  63.80689904999999


/opt/conda/envs/python35-paddle120-env/lib/python3.5/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/envs/python35-paddle120-env/lib/python3.5/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


0       0.249910
50      0.180633
100     0.192535
150     0.253337
200     0.156103
250     0.185722
300     0.207306
350     0.166525
400     0.115783
450     0.140239
500     0.093649
550     0.134033
600     0.263693
650     0.165918
700     0.436844
750     0.197838
800     0.312173
850     0.161930
900     0.156833
950     0.179383
1000    0.197355
1050    0.178522
1100    0.166609
1150    0.169956
1200    0.127062
1250    0.178387
1300    0.355139
1350    0.178541
1400    0.355064
1450    0.168823
          ...   
4500    0.250614
4550    0.160334
4600    0.181724
4650    0.202574
4700    0.227717
4750    0.173334
4800    0.135898
4850    0.167044
4900    0.191231
4950    0.162354
5000    0.088339
5050    0.200507
5100    0.277622
5150    0.152901
5200    0.250391
5250    0.177016
5300    0.083524
5350    0.214081
5400    0.198213
5450    0.209733
5500    0.105927
5550    0.181194
5600    0.185484
5650    0.191894
5700    0.201376
5750    0.165132
5800    0.085453
5850    0.2025

In [20]:
# grid_search('xgb')

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV] min_child_weight=1, n_estimators=1000, max_depth=5, colsample_bytree=0.5 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  min_child_weight=1, n_estimators=1000, max_depth=5, colsample_bytree=0.5, total=  11.4s
[CV] min_child_weight=1, n_estimators=1000, max_depth=5, colsample_bytree=0.5 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   11.4s remaining:    0.0s


[CV]  min_child_weight=1, n_estimators=1000, max_depth=5, colsample_bytree=0.5, total=  10.9s
[CV] min_child_weight=1, n_estimators=1000, max_depth=5, colsample_bytree=0.5 
[CV]  min_child_weight=1, n_estimators=1000, max_depth=5, colsample_bytree=0.5, total=  11.0s
[CV] min_child_weight=1, n_estimators=1000, max_depth=5, colsample_bytree=0.5 
[CV]  min_child_weight=1, n_estimators=1000, max_depth=5, colsample_bytree=0.5, total=  10.9s
[CV] min_child_weight=1, n_estimators=1000, max_depth=5, colsample_bytree=0.5 
[CV]  min_child_weight=1, n_estimators=1000, max_depth=5, colsample_bytree=0.5, total=  11.0s
[CV] min_child_weight=1, n_estimators=2000, max_depth=5, colsample_bytree=0.5 
[CV]  min_child_weight=1, n_estimators=2000, max_depth=5, colsample_bytree=0.5, total=  21.9s
[CV] min_child_weight=1, n_estimators=2000, max_depth=5, colsample_bytree=0.5 
[CV]  min_child_weight=1, n_estimators=2000, max_depth=5, colsample_bytree=0.5, total=  22.1s
[CV] min_child_weight=1, n_estimators=200

[Parallel(n_jobs=1)]: Done 480 out of 480 | elapsed: 312.7min finished
/opt/conda/envs/python35-paddle120-env/lib/python3.5/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


best parameters: {'min_child_weight': 4, 'n_estimators': 1000, 'max_depth': 5, 'colsample_bytree': 0.5}
best score: -0.6619330762458744


请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 